In [2]:
import pandas as pd
import json
import requests
import math
import numpy as np

with open('config.json') as f:
    settings = json.load(f)
df = pd.read_csv("block.csv")

# Getting the distance and time Matrix
Should only be run once and the output is saved to reduce call to Google api.

In [46]:
release_pts = pd.read_excel('Yishun Wolbachia Release Points.xlsx')

In [47]:
release_pts['NumRelease'] = release_pts['ReleaseLocation'].str.split(',').apply(len)
release_pts['Block'] = release_pts['Block'].astype('str')

In [48]:
release_pts['Block'] = release_pts['Block'].str.replace('.Bin Centre', '')

In [49]:
release_pts

,Location,Postal Code,Block,ReleaseLocation,NumRelease
0,YISHUN STREET 21,760201.0,201,"51,53",2
1,YISHUN STREET 21,760201.0,201,"55,57",2
2,YISHUN STREET 21,760201.0,201,"59,61",2
3,YISHUN STREET 21,760201.0,201,"63,65",2
4,YISHUN STREET 21,760201.0,201,"67,69",2
5,YISHUN STREET 21,760202.0,202,"73,75",2
6,YISHUN STREET 21,760202.0,202,"77,79",2
7,YISHUN STREET 21,760202.0,202,"81,83",2
8,YISHUN STREET 21,760202.0,202,"85,87",2
9,YISHUN STREET 21,760202.0,202,"89,91",2


In [63]:
def count_release(df):
    output = {
        'Road' : df.iloc[0,0],
        'NumRelease' : df['NumRelease'].sum(),
    }
    return pd.Series(output)

In [64]:
df = release_pts.groupby('Block').apply(count_release).reset_index()

In [66]:
df['address'] = df['Block'] + '+' + df['Road'].str.replace(' ','+')

In [75]:
df['location'] = None

In [76]:
url= "https://maps.googleapis.com/maps/api/geocode/json"

for each in range(len(df)):
    if not df.loc[each, 'location']:
        params = dict(
            address = df.loc[each, 'address'],
            key = settings['api_key']
        )

        resp = requests.get(url=url, params=params)
        data = resp.json()

        try:
            df.loc[each, 'location'] = str(data['results'][0]['geometry']['location']['lat']) + "," + str(data['results'][0]['geometry']['location']['lng'])
        except:
            print(each)
            print(data)

In [79]:
df.to_pickle('block_with_geoloc.pickle')

In [80]:
arr,arr1,arr2,arr3,arr4 = "","", "", "", ""

arr_total = []

for index, row in df.iterrows():
    arr += str(row['address']) + "|"

for index, row in df.iterrows():
    if(index < 25):
        arr1 += str(row['address']) + "|"
        
for index, row in df.iterrows():
    if(25 <= index and index < 50):
        arr2 += str(row['address']) + "|"
        
for index, row in df.iterrows():
    if(50 <= index and index < 75):
        arr3 += str(row['address']) + "|"
    
    
for index, row in df.iterrows():
    if(75 <= index and index < 84):
        arr4 += str(row['address']) + "|"
        
arr_total.append(arr1[:-1])
arr_total.append(arr2[:-1])
arr_total.append(arr3[:-1])
arr_total.append(arr4[:-1])

arr_total

['201+YISHUN+STREET+21|202+YISHUN+STREET+21|203+YISHUN+STREET+21|204+YISHUN+STREET+21|205+YISHUN+STREET+21|206+YISHUN+STREET+21|207+YISHUN+STREET+21|208+YISHUN+STREET+21|209+YISHUN+STREET+21|210+YISHUN+STREET+21|211+YISHUN+STREET+21|212+YISHUN+STREET+21|213+YISHUN+STREET+21|214+YISHUN+STREET+21|215+YISHUN+STREET+21|216+YISHUN+STREET+21|217+YISHUN+STREET+21|218+YISHUN+STREET+21|219+YISHUN+STREET+21|220+YISHUN+STREET+21|221+YISHUN+STREET+21|222+YISHUN+STREET+21|223+YISHUN+STREET+21|224+YISHUN+STREET+21|225+YISHUN+STREET+21',
 '226+YISHUN+STREET+21|227+YISHUN+STREET+21|228+YISHUN+STREET+21|229+YISHUN+STREET+21|230+YISHUN+STREET+21|231+YISHUN+STREET+21|232+YISHUN+STREET+21|233+YISHUN+STREET+21|234+YISHUN+STREET+21|235+YISHUN+STREET+21|236+YISHUN+RING+ROAD|237+YISHUN+RING+ROAD|238+YISHUN+RING+ROAD|239+YISHUN+RING+ROAD|240+YISHUN+RING+ROAD|241+YISHUN+RING+ROAD|242+YISHUN+RING+ROAD|243+YISHUN+RING+ROAD|244+YISHUN+RING+ROAD|245+YISHUN+AVENUE+9|246+YISHUN+AVENUE+9|247+YISHUN+AVENUE+9|248+YISHUN

In [82]:
# WILL RUN FOR 2 MINS AT MAX

time_matrix = np.zeros((84,84))
time_array = []
dist_matrix = np.zeros((84,84))
dist_array = []

url_google_route = "https://maps.googleapis.com/maps/api/distancematrix/json"


for each_index in range(len(df)):
    time_array = []
    dist_array = []
    print("Inititaing " + str(each_index) + "/83 first...")
    for index, dest_input in enumerate(arr_total):
#         print("Working on " + str(index) + "/3 first...")
#         print(dest_input)
        params = dict(
            destinations = dest_input,
            origins = df.iloc[each_index]['address'],
            key = settings['api_key'],
            mode = "walking"
        )
        resp = requests.get(url=url_google_route, params=params)
        data = resp.json()
        for each in range(len(data['rows'][0]['elements'])):
            if data['rows'][0]['elements'][each]['status'] == 'OK':
                time_array.append(data['rows'][0]['elements'][each]['duration']['value'])
                dist_array.append(data['rows'][0]['elements'][each]['distance']['value'])
            else:
                print(each)
                print("Origin: {}, Destination: ".format(data['origin_addresses'][0]))
                print(data)
    print(each_index)
    time_matrix[each_index] = time_array
    dist_matrix[each_index] = dist_array


Inititaing 0/83 first...
0
Inititaing 1/83 first...
1
Inititaing 2/83 first...
2
Inititaing 3/83 first...
3
Inititaing 4/83 first...
4
Inititaing 5/83 first...
5
Inititaing 6/83 first...
6
Inititaing 7/83 first...
7
Inititaing 8/83 first...
8
Inititaing 9/83 first...
9
Inititaing 10/83 first...
10
Inititaing 11/83 first...
11
Inititaing 12/83 first...
12
Inititaing 13/83 first...
13
Inititaing 14/83 first...
14
Inititaing 15/83 first...
15
Inititaing 16/83 first...
16
Inititaing 17/83 first...
17
Inititaing 18/83 first...
18
Inititaing 19/83 first...
19
Inititaing 20/83 first...
20
Inititaing 21/83 first...
21
Inititaing 22/83 first...
22
Inititaing 23/83 first...
23
Inititaing 24/83 first...
24
Inititaing 25/83 first...
25
Inititaing 26/83 first...
26
Inititaing 27/83 first...
27
Inititaing 28/83 first...
28
Inititaing 29/83 first...
29
Inititaing 30/83 first...
30
Inititaing 31/83 first...
31
Inititaing 32/83 first...
32
Inititaing 33/83 first...
33
Inititaing 34/83 first...
34
Initi

In [84]:
np.save('time_matrix.npy', time_matrix)
np.save('dist_matrix.npy', dist_matrix)

# Loading the processed data

In [17]:
time_matrix = np.load('time_matrix.npy')
dist_matrix = np.load('dist_matrix.npy')
df = pd.read_pickle('block_with_geoloc.pickle')

In [18]:
time_matrix

array([[  0.,  42., 502., ..., 748., 716., 557.],
       [ 40.,   0., 543., ..., 789., 756., 597.],
       [504., 546.,   0., ..., 941., 908., 749.],
       ...,
       [753., 796., 944., ...,   0., 132., 185.],
       [730., 772., 920., ..., 141.,   0., 137.],
       [592., 634., 783., ..., 216., 159.,   0.]])

In [19]:
dist_matrix

array([[   0.,   57.,  689., ..., 1009.,  975.,  777.],
       [  57.,    0.,  746., ..., 1065., 1032.,  834.],
       [ 689.,  746.,    0., ..., 1274., 1240., 1043.],
       ...,
       [1009., 1065., 1274., ...,    0.,  182.,  270.],
       [ 975., 1032., 1240., ...,  182.,    0.,  198.],
       [ 777.,  834., 1043., ...,  270.,  198.,    0.]])

In [88]:
df

,Block,Road,NumRelease,address,location
0,201,YISHUN STREET 21,10,201+YISHUN+STREET+21,"1.4307489,103.8378901"
1,202,YISHUN STREET 21,10,202+YISHUN+STREET+21,"1.4307641,103.8372645"
2,203,YISHUN STREET 21,10,203+YISHUN+STREET+21,"1.4307836,103.8363356"
3,204,YISHUN STREET 21,10,204+YISHUN+STREET+21,"1.4306109,103.836163"
4,205,YISHUN STREET 21,10,205+YISHUN+STREET+21,"1.4311456,103.8361002"
5,206,YISHUN STREET 21,2,206+YISHUN+STREET+21,"1.4312278,103.8367514"
6,207,YISHUN STREET 21,2,207+YISHUN+STREET+21,"1.431274,103.837047"
7,208,YISHUN STREET 21,10,208+YISHUN+STREET+21,"1.4309044,103.8376046"
8,209,YISHUN STREET 21,8,209+YISHUN+STREET+21,"1.43148,103.8377984"
9,210,YISHUN STREET 21,10,210+YISHUN+STREET+21,"1.4313397,103.8384973"


In [20]:
time_matrix[:,37] = 0 # Setting return to base time to zero.

In [4]:
# Adding in the additional time for release
for idx, value in df['NumRelease'].iteritems():
    time_matrix[idx, :] += value*60

In [90]:
time_matrix

array([[ 600.,  642., 1102., ..., 1348., 1316., 1157.],
       [ 640.,  600., 1143., ..., 1389., 1356., 1197.],
       [1104., 1146.,  600., ..., 1541., 1508., 1349.],
       ...,
       [1473., 1516., 1664., ...,  720.,  852.,  905.],
       [1450., 1492., 1640., ...,  861.,  720.,  857.],
       [1072., 1114., 1263., ...,  696.,  639.,  480.]])

In [21]:
"""Vehicles Routing Problem (VRP)."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = time_matrix
    data['num_vehicles'] = 6
    data['depot'] = 37
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))




def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data['distance_matrix']), data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        100000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == '__main__':
    main()

Route for vehicle 0:
 37 ->  41 ->  39 ->  40 ->  10 ->  9 ->  0 ->  1 ->  7 ->  8 ->  6 ->  11 ->  12 ->  15 ->  16 ->  14 ->  13 ->  4 ->  5 ->  2 -> 37
Distance of the route: 1550m

Route for vehicle 1:
 37 ->  36 ->  56 ->  63 ->  58 ->  57 ->  59 ->  60 ->  61 ->  62 ->  68 ->  67 ->  64 ->  65 -> 37
Distance of the route: 1381m

Route for vehicle 2:
 37 ->  35 ->  38 ->  79 ->  80 ->  83 ->  82 ->  81 ->  78 ->  77 ->  76 ->  75 ->  74 ->  73 ->  3 -> 37
Distance of the route: 1663m

Route for vehicle 3:
 37 ->  55 ->  53 ->  54 ->  43 ->  42 ->  44 ->  46 ->  45 ->  47 ->  49 ->  52 ->  51 ->  50 ->  48 -> 37
Distance of the route: 1292m

Route for vehicle 4:
 37 ->  27 ->  26 ->  29 ->  25 ->  24 ->  69 ->  70 ->  71 ->  72 ->  66 -> 37
Distance of the route: 1308m

Route for vehicle 5:
 37 ->  32 ->  23 ->  19 ->  22 ->  21 ->  20 ->  17 ->  18 ->  31 ->  30 ->  28 ->  33 ->  34 -> 37
Distance of the route: 1513m

Maximum of the route distances: 1663m


# Plotting the routes 

In [22]:
"""Solve the CVRP problem."""
# Instantiate the data problem.
data = create_data_model()

# Create the routing index manager.
manager = pywrapcp.RoutingIndexManager(
    len(data['distance_matrix']), data['num_vehicles'], data['depot'])

# Create Routing Model.
routing = pywrapcp.RoutingModel(manager)


# Create and register a transit callback.
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

# Define cost of each arc.
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

# Add Distance constraint.
dimension_name = 'Distance'
routing.AddDimension(
    transit_callback_index,
    0,  # no slack
    100000,  # vehicle maximum travel distance
    True,  # start cumul to zero
    dimension_name)
distance_dimension = routing.GetDimensionOrDie(dimension_name)
distance_dimension.SetGlobalSpanCostCoefficient(100)

# Setting first solution heuristic.
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)

# Print solution on console.
if solution:
    print_solution(data, manager, routing, solution)

Route for vehicle 0:
 37 ->  41 ->  39 ->  40 ->  10 ->  9 ->  0 ->  1 ->  7 ->  8 ->  6 ->  11 ->  12 ->  15 ->  16 ->  14 ->  13 ->  4 ->  5 ->  2 -> 37
Distance of the route: 1550m

Route for vehicle 1:
 37 ->  36 ->  56 ->  63 ->  58 ->  57 ->  59 ->  60 ->  61 ->  62 ->  68 ->  67 ->  64 ->  65 -> 37
Distance of the route: 1381m

Route for vehicle 2:
 37 ->  35 ->  38 ->  79 ->  80 ->  83 ->  82 ->  81 ->  78 ->  77 ->  76 ->  75 ->  74 ->  73 ->  3 -> 37
Distance of the route: 1663m

Route for vehicle 3:
 37 ->  55 ->  53 ->  54 ->  43 ->  42 ->  44 ->  46 ->  45 ->  47 ->  49 ->  52 ->  51 ->  50 ->  48 -> 37
Distance of the route: 1292m

Route for vehicle 4:
 37 ->  27 ->  26 ->  29 ->  25 ->  24 ->  69 ->  70 ->  71 ->  72 ->  66 -> 37
Distance of the route: 1308m

Route for vehicle 5:
 37 ->  32 ->  23 ->  19 ->  22 ->  21 ->  20 ->  17 ->  18 ->  31 ->  30 ->  28 ->  33 ->  34 -> 37
Distance of the route: 1513m

Maximum of the route distances: 1663m


In [12]:
for vehicle_id in range(data['num_vehicles']):
    loc = []
    index = routing.Start(vehicle_id)
    print('\nRoute for vehicle {}:'.format(vehicle_id))
    route_distance = 0
    tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
    loc.append((float(tmp[0]), float(tmp[1])))
    while not routing.IsEnd(index):
        plan_output += ' {} -> '.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
        loc.append((float(tmp[0]), float(tmp[1])))
    print(loc)
        


Route for vehicle 0:
[(1.4332785, 103.8392453), (1.4321146, 103.83922), (1.4329338, 103.840239), (1.4341163, 103.8388803), (1.4337854, 103.8396007), (1.4353817, 103.8385457), (1.4358023, 103.8391521), (1.4361929, 103.8390278), (1.4363967, 103.8379745), (1.4370249, 103.8372726), (1.4359653, 103.8365253), (1.4361319, 103.8346631), (1.436381, 103.834464), (1.4359296, 103.8352515), (1.4353741, 103.8354275), (1.4332785, 103.8392453)]

Route for vehicle 1:
[(1.4332785, 103.8392453), (1.4306109, 103.836163), (1.4346738, 103.8351369), (1.435152, 103.836505), (1.4346557, 103.8365966), (1.4340405, 103.8363799), (1.433338, 103.83597), (1.433043, 103.836044), (1.432693, 103.836157), (1.4341706, 103.8370155), (1.434254, 103.8371287), (1.434889, 103.836926), (1.4345403, 103.8385407), (1.4344091, 103.8376494), (1.4332785, 103.8392453)]

Route for vehicle 2:
[(1.4332785, 103.8392453), (1.4353956, 103.8375364), (1.4350013, 103.8381557), (1.4340362, 103.8399673), (1.433955, 103.8412354), (1.4343073, 10

In [23]:
import folium
color = ["red", "blue", "green", "yellow", "black", "grey"]
mapa = folium.Map([1.3,103.9],
                  zoom_start=4,
                  tiles='cartodbpositron')
for vehicle_id in range(data['num_vehicles']):
    loc = []
    index = routing.Start(vehicle_id)
    print('\nRoute for vehicle {}:'.format(vehicle_id))
    route_distance = 0
    tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
    loc.append((float(tmp[0]), float(tmp[1])))
    while not routing.IsEnd(index):
        plan_output += ' {} -> '.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        tmp = df.loc[manager.IndexToNode(index), 'location'].split(',')
        loc.append((float(tmp[0]), float(tmp[1])))
    folium.PolyLine(loc, color=color[vehicle_id], weight=2.5, opacity=1).add_to(mapa)
mapa


Route for vehicle 0:

Route for vehicle 1:

Route for vehicle 2:

Route for vehicle 3:

Route for vehicle 4:

Route for vehicle 5:
